In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

In [3]:
train_test = utils.read_df_pkl('../input/base_term*0*').set_index(key)
df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:00<00:00, 78.83it/s]


SystemExit: 

In [4]:
col_term = 'hist_regist_term'

train_test['diff_new_hist_regist_term'] = train_test['new_regist_term'] - train_test[col_term]
train_test[col_term] = train_test[col_term].map(lambda x:
                                                                    24 if x>=24 else
#                                                                     18 if x>=18 else
#                                                                     15 if x>=15 else
#                                                                     12 if 9<=x and x<=14 else
#                                                                     6 if 6<=x and x<=8  else 
#                                                                     5 if x==5 else
#                                                                     4 if x==4 else
#                                                                     3 if x<=3 
                                                                    x
                                                                   )
train_test['new_regist_term'] = train_test['new_regist_term'].map(lambda x:
                                                                    24 if x>=24 else
#                                                                     18 if x>=18 else
#                                                                     15 if x>=15 else
#                                                                     12 if x>=12 else
                                                                    x
                                                                   )

utils.to_df_pkl(df=train_test.reset_index(), path='../input', fname='base_term')

3it [00:00, 32.50it/s]

shape: (325540, 8)


In [10]:
df_hist.set_index(key, inplace=True)
df_new.set_index(key, inplace=True)
term_cols = [col for col in train_test.columns if col.count('term')]
df_hist = df_hist.join(train_test[term_cols])
df_new = df_new.join(train_test[term_cols])
df_new.head()

,authorized_flag,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,first_active_month,all_term,yyyymm,yyyymmdd,weekofyear,yyyy_week,purchase_amount_over,hist_personal_term,hist_regist_term,new_personal_term,new_regist_term,diff_new_hist_regist_term
card_id,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,1,76,0,1.0,B,222,M_ID_08f01305af,2,0.075195,2018-04-03 11:13:35,3,2,21,2017-02-01,454,2018-04,2018-04-03,14,2018-14,0.0,13,12.0,1.0,15.0,3.0
C_ID_00007093c1,1,69,0,1.0,B,879,M_ID_00a6ca8a8a,2,0.090332,2018-04-09 16:23:59,1,9,29,2017-02-01,454,2018-04,2018-04-09,15,2018-15,0.0,13,12.0,1.0,15.0,3.0
C_ID_0001238066,1,314,0,1.0,B,307,M_ID_7d8102bb34,1,0.014168,2018-03-01 16:48:27,1,9,19,2017-09-01,242,2018-03,2018-03-01,9,2018-09,0.0,6,6.0,2.0,8.0,2.0
C_ID_0001238066,1,314,0,1.0,B,367,M_ID_235e546dcc,1,0.074707,2018-03-03 22:44:57,1,9,16,2017-09-01,242,2018-03,2018-03-03,9,2018-09,0.0,6,6.0,2.0,8.0,2.0
C_ID_0001238066,1,333,0,1.0,B,783,M_ID_a88790a464,1,0.105469,2018-03-04 13:05:16,1,9,19,2017-09-01,242,2018-03,2018-03-04,9,2018-09,0.0,6,6.0,2.0,8.0,2.0


In [13]:
df_hist['category_3'] = df_hist['category_3'].map(lambda x: 1 if x=='B' else 0)
df_new['category_3'] = df_new['category_3'].map(lambda x: 1 if x=='B' else 0)

In [19]:
# hist , newからauth, category_1, category_3_Bの情報をとってくる

df_list = [df_hist, df_new]
fname_list = ['new', 'hist']

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df, fname in zip(df_list, fname_list):

    aggs = {}
    agg_cols = [key, 'authorized_flag', 'category_1', 'category_3', 'purchase_amount', 'purchase_date']
    if fname.count('hist'):
        aggs['authorized_flag'] = ['mean']
    aggs['category_1'] = ['mean']
    aggs['category_3'] = ['mean']
    aggs['purchase_amount'] = ['sum']
    aggs['purchase_date'] = ['size', 'min']
    
    df_agg = df.reset_index()[agg_cols].groupby(key).agg(aggs)
    new_cols = get_new_columns(fname, aggs)
    df_agg.columns = new_cols
    
    train_test = train_test.join(df_agg)

In [24]:
train_test['first_buy'] = (train_test['hist_purchase_date_min'] - train_test['first_active_month']).dt.days//30
train_test['first_buy'] = train_test['first_buy'].map(lambda x:
                                                      24 if x>=24 else x )
train_test['first_buy'].value_counts()

24.0    31563
5.0     23851
4.0     23815
6.0     22849
7.0     21020
3.0     20233
8.0     18092
9.0     15356
10.0    13305
11.0    12098
12.0    11024
14.0    10895
13.0    10491
15.0    10227
16.0     8600
17.0     6835
19.0     5746
18.0     5522
20.0     5103
21.0     4636
22.0     4061
23.0     3420
2.0      1235
1.0        23
0.0         1
Name: first_buy, dtype: int64

In [115]:
# fam = 'first_active_month'
# base[base[col_term]==5]['first_active_month'].value_counts()
# base[base['first_active_month']=='2017-10-01'][target].map(lambda x: np.round(x, 1)).value_counts().sum()
# base['first_active_month'].value_counts()
# base[base[col_term]==4][target].min()
base = train_test.reset_index()

out = 'outlier'
base['outlier'] = base[target].map(lambda x: 1 if x<-30 else 0)
df_fam_term = base[[key, fam, col_term]].groupby([fam, col_term]).size().reset_index().rename(columns={0:'cnt'})
df_fam = base[[key, fam, col_term]].groupby([fam]).size().reset_index().rename(columns={0:'max_cnt'})
df_out = base[[fam, col_term, out]].groupby([fam, col_term])[out].sum().reset_index().rename(columns={out:'out_cnt'})
df_loy_max = base[[fam, col_term, target]].groupby([fam, col_term])[target].max().reset_index().rename(columns={target:'loy_max'})
df_loy_min = base[[fam, col_term, target]].groupby([fam, col_term])[target].min().reset_index().rename(columns={target:'loy_min'})
df = df_fam_term.merge(df_fam, on=fam, how='inner').merge(df_out, on=[fam, col_term], how='inner').merge(df_loy_max, on=[fam, col_term], how='inner').merge(df_loy_min, on=[fam, col_term], how='inner')
df['cnt_ratio'] = df['cnt'] / df['max_cnt']
df['out_ratio'] = df['out_cnt'] / df['cnt']
df.to_csv('../output/0208_elo_fam_hist_regist_term_record.csv', index=False)

In [15]:
term_name = 'new_regist_term'
term_list = sorted(list(train_test[term_name].value_counts().index))

feat_cols = [col for col in train_test if col.count('new_cat')]
# first_buyのlift
# feat_cols = ['first_buy', 'new_first_buy']

lift_list = []
for term in tqdm(term_list):
    
    tmp_list = []
    for col in feat_cols:
        tmp = train_test[train_test[term_name]==term].reset_index()
        tmp = tmp[[key, col]]
        base_avg = tmp[col].mean()
        tmp.set_index(key, inplace=True)
        tmp[col] /= base_avg
#         tmp.rename(columns={col:f'lift_regist_term{int(term)}_{col}'}, inplace=True)
        tmp.rename(columns={col:f'lift_{term_name}_{col}'}, inplace=True)
        tmp_list.append(tmp)
    df_term = pd.concat(tmp_list, axis=1)
    lift_list.append(df_term)
    
df_lift = pd.concat(lift_list, axis=0)
print(df_lift.shape)
base = utils.read_df_pkl('../input/base_term*0*').set_index(key)['target'].to_frame()
df_lift = base.join(df_lift)
df_lift.head()
# df_lift['diff_lift_regist_term_new_first_buy_-_first_buy'] = df_lift['lift_regist_term_new_first_buy'] - df_lift['lift_regist_term_first_buy']

100%|██████████| 11/11 [00:17<00:00,  1.76s/it]

(290001, 24)


,lift_new_regist_term_new_cat1_subsector_id_nunique,lift_new_regist_term_new_cat1_merchant_id_nunique,lift_new_regist_term_new_cat1_merchant_category_id_nunique,lift_new_regist_term_new_cat1_yyyymmdd_nunique,lift_new_regist_term_new_cat1_yyyy_week_nunique,lift_new_regist_term_new_cat1_purchase_amount_sum,lift_new_regist_term_new_cat1_installments_mean,lift_new_regist_term_new_cat1_amount_per_installments_sum,lift_new_regist_term_new_cat1_month_lag_max,lift_new_regist_term_new_cat1_month_lag_mean,lift_new_regist_term_new_cat1_month_diff_mean,lift_new_regist_term_new_cat1_card_id_size,lift_new_regist_term_new_cat0_subsector_id_nunique,lift_new_regist_term_new_cat0_merchant_id_nunique,lift_new_regist_term_new_cat0_merchant_category_id_nunique,lift_new_regist_term_new_cat0_yyyymmdd_nunique,lift_new_regist_term_new_cat0_yyyy_week_nunique,lift_new_regist_term_new_cat0_purchase_amount_sum,lift_new_regist_term_new_cat0_installments_mean,lift_new_regist_term_new_cat0_amount_per_installments_sum,lift_new_regist_term_new_cat0_month_lag_max,lift_new_regist_term_new_cat0_month_lag_mean,lift_new_regist_term_new_cat0_month_diff_mean,lift_new_regist_term_new_cat0_card_id_size
card_id,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_b6302b31c6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.951036,0.687580,0.923600,0.764075,1.000639,0.576172,0.000000,1.126953,1.123779,0.949531,1.219330,0.687310
C_ID_1919f9080e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.380414,0.229193,0.307867,0.305630,0.500320,0.257812,1.768555,0.204834,0.561889,0.712148,0.904665,0.229103
C_ID_09da8a2858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.521657,1.604353,2.001133,1.833779,1.751118,4.917969,3.429688,1.494141,1.123779,1.119090,0.882188,1.603723
C_ID_4e2e2c0822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.331450,0.916773,1.231466,0.916890,1.250799,1.378906,0.000000,2.697266,1.123779,1.157240,0.875165,0.916413
C_ID_951ae2900b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.380414,0.458387,0.615733,0.458445,0.500320,0.613770,2.357422,0.397705,0.561889,0.712148,0.904665,0.458206
